# JSONification

In [1]:
from pathlib import Path
from polymerist.genutils.fileutils import assemble_path


OUTPUT_DIR = Path('scratch_misc') # dummy directory for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

JSONIFY_DIR = OUTPUT_DIR / 'jsonifiables'
JSONIFY_DIR.mkdir(exist_ok=True)

Peppered throughout the accompanying polymer demos, you may have noticed examples of container classes which support convenient obj.to_file(...) and cls.from_file(...) operation  
For instance, consider the MonomerGroup class in the [polymerization demos](../1-polymerization/1.0-index.ipynb) or the many containers in the [simulation parameters demo](../3-workflows/3.2-serializable_simulation_parameters.ipynb)

These are examples of the much more general JSONifiable type `polymerist` defines, subclasses of which all support obj.to_file(...) and cls.from_file(...) methods natively  
The `jsonify` module of polymerist allows you to detect such JSONifiable containers, as well as create your own from an arbitrary dataclass!

In [2]:
from polymerist.mdtools.openmmtools.parameters import SimulationParameters
from polymerist.genutils.fileutils.jsonio.jsonify import JSONifiable, make_jsonifiable


print(issubclass(SimulationParameters, JSONifiable)) # SimulationParameters natively supports to_file() and from_file methods()
print(issubclass(list, JSONifiable)) # the builtin list (and other builtin classes) do not

True
False


For disclosure, here are all of the JSONifiable classes within all of `polymerist`  at the time of writing:

In [3]:
from polymerist.mdtools.openmmtools.parameters import (
    ThermoParameters,
    IntegratorParameters,
    ReporterParameters,
    SimulationParameters,
)
from polymerist.polymers.monomers import MonomerGroup
from polymerist.polymers.building.sequencing import LinearCopolymerSequencer
from polymerist.mdtools.openfftools.partialcharge.rescharge.rctypes import ChargesByResidue


jsonifiable_classes_in_polymerist : list[JSONifiable] = [
    ThermoParameters,
    IntegratorParameters,
    ReporterParameters,
    SimulationParameters,
    MonomerGroup,
    LinearCopolymerSequencer,
    ChargesByResidue,
]
print( all(issubclass(cls, JSONifiable) for cls in jsonifiable_classes_in_polymerist) )

[22:35:00] WARNING: not removing hydrogen atom with dummy atom neighbors


True


### Making your own JSONifiables!
Basically any* Python [dataclass](https://docs.python.org/3/library/dataclasses.html) can be instantly turned JSONifiable with the `make_jsonifiable` decorator `polymerist` provides you:
<a id='asterisk'></a>

In [4]:
from dataclasses import dataclass, field
from polymerist.genutils.fileutils.jsonio.jsonify import make_jsonifiable


@make_jsonifiable # this comes from polymerist and is literally all you need to do yourself!
@dataclass # this is a Python builtin
class Student:
    name   : str
    age    : int
    grades : dict[str, float] = field(default_factory=dict)

student = Student(
    name='Drew A. Blanc',
    age=24,
    grades={
        'chemistry' : 91.7,
        'mathematics' : 99.8,
        'physics' : 88.3,
        'english' : 78.4,
    })
print(student)

Student(name='Drew A. Blanc', age=24, grades={'chemistry': 91.7, 'mathematics': 99.8, 'physics': 88.3, 'english': 78.4})


In [5]:
student_path = assemble_path(JSONIFY_DIR, 'student', extension='json')
student.to_file(student_path) # this works out-of-box

In [6]:
recorded_student = Student.from_file(student_path)
print(recorded_student.name, recorded_student.grades)

Drew A. Blanc {'chemistry': 91.7, 'mathematics': 99.8, 'physics': 88.3, 'english': 78.4}


#### *Adapting nonstandard attribute types for JSON serialization
C.f. the [asterisk above](#asterisk
), it may be that the container you wish to serialize has fields which are not, by default, JSON-serializable types, shown below

In [11]:
from polymerist.genutils.fileutils.jsonio.serialize import JSONSerializable

print(JSONSerializable) # these are the ONLY types which, by default, can be faithfully read from and written to a JSON file

typing.Union[str, bool, int, float, tuple, list, dict]


For example, the below example <font color='red'>will fail and raise TypeError</font> due to the presence of Path and Quantity attribute values

In [12]:
from pathlib import Path
from openmm.unit import Quantity, second


@make_jsonifiable
@dataclass
class LoggingParameters:
    walltime : Quantity
    logfile : Path = field(default_factory=lambda: Path('simulation.log'))
    
log_params = LoggingParameters(30*second)
log_params.to_file(JSONIFY_DIR / 'log_params.json')

TypeError: Object of type Quantity is not JSON serializable

This is easily overcome by providing "TypeSerializer" definitions to make_jsonifiable, which define how ordinarily non-JSON-serializable types should be encoded and decoded

In [13]:
from polymerist.genutils.fileutils.jsonio.serialize import PathSerializer, QuantitySerializer, MultiTypeSerializer


@make_jsonifiable(type_serializer=MultiTypeSerializer(PathSerializer, QuantitySerializer)) # bundle multiple serializers together
@dataclass
class LoggingParameters:
    walltime : Quantity
    logfile : Path = field(default_factory=lambda: Path('simulation.log'))
    
log_params = LoggingParameters(30*second)
log_params_path = assemble_path(JSONIFY_DIR, 'log_params', extension='json')
log_params.to_file(log_params_path)

In [14]:
log_params_from_file = LoggingParameters.from_file(log_params_path)
print(log_params_from_file.walltime, type(log_params_from_file.walltime))
print(log_params_from_file.logfile, type(log_params_from_file.logfile))

30 s <class 'openmm.unit.quantity.Quantity'>
simulation.log <class 'pathlib.PosixPath'>


By default, PathSerializer and QuantitySerializer are shipped with `polymerist`; defining your own, however, is not especially difficult  
Simply inherit from the base TypeSerializer and provide a target type and encode/decode implementations

In [15]:
import numpy as np
from typing import Any
from polymerist.genutils.fileutils.jsonio.serialize import TypeSerializer


class NDArraySerializer(TypeSerializer, python_type=np.ndarray):
    '''For handling JSON serialization of numpy arrays'''
    @staticmethod
    def encode(python_obj : np.ndarray[Any]) -> list[Any]:
        return python_obj.tolist()
    
    @staticmethod
    def decode(value : list[Any]) -> np.ndarray[Any]:
        return np.array(value)

In [16]:
@make_jsonifiable(type_serializer=NDArraySerializer)
@dataclass
class EulerRotation:
    '''Euler-angle description of a rotation by its axis and angle around the axis'''
    angle_rad : float = 0.0 # NOTE: opted not to make this a Quantity (though in practice your should) to show JUST the effects of the array serializer
    axis      : np.ndarray[float] = field(default_factory=lambda: np.array([1.0, 0.0, 0.0]))
    
rotation = EulerRotation(angle_rad=np.pi/3, axis=np.array([0.0, 1.0, 0.0]))
rotation.to_file(assemble_path(JSONIFY_DIR, 'rotation', extension='json'))


In [17]:
rotation_from_file = EulerRotation.from_file(JSONIFY_DIR/'rotation.json')
print(rotation_from_file)
print(type(rotation_from_file.axis))

EulerRotation(angle_rad=1.0471975511965976, axis=array([0., 1., 0.]))
<class 'numpy.ndarray'>
